In [181]:
import pandas as pd

In [182]:
locktime_raw = pd.read_excel('TraiCay_LockTime.xlsx')
locktime_raw.head()

,Store_Name,Store,Tuesday,Tuesday.1,Wednesday,Wednesday.1,Thursday,Thursday.1,Friday,Friday.1,Saturday,Saturday.1,Sunday,Sunday.1,Monday,Monday.1
0,C.Quynh,151,11:00:00,1D,11:00:00,1D,11:00:00,1D,11:00:00,1D,11:00:00,1D,11:15:00,1D,11:15:00,2D
1,ND.Chieu,155,11:00:00,1D,11:00:00,1D,11:00:00,1D,11:00:00,1D,11:00:00,1D,11:15:00,1D,11:15:00,2D
2,XLH.Noi,161,11:00:00,1D,11:00:00,1D,11:00:00,1D,11:00:00,1D,11:00:00,1D,11:15:00,1D,11:15:00,2D
3,LT.Kiet,505,11:00:00,1D,11:00:00,1D,11:00:00,1D,11:00:00,1D,11:00:00,1D,11:15:00,1D,11:15:00,2D
4,CM HT.Phat,159,11:00:00,1D,11:00:00,1D,11:00:00,1D,11:00:00,1D,11:00:00,1D,11:15:00,1D,11:15:00,2D


In [183]:
locktime_lockbefore = locktime_raw.drop(columns=locktime_raw.columns[2::2])
locktime_lockhour = locktime_raw.drop(columns=locktime_raw.columns[3::2])

In [184]:
for col in locktime_lockbefore.columns[2:]:
    locktime_lockbefore = locktime_lockbefore.rename(columns={col:col[:-2]})
locktime_lockbefore.head()

,Store_Name,Store,Tuesday,Wednesday,Thursday,Friday,Saturday,Sunday,Monday
0,C.Quynh,151,1D,1D,1D,1D,1D,1D,2D
1,ND.Chieu,155,1D,1D,1D,1D,1D,1D,2D
2,XLH.Noi,161,1D,1D,1D,1D,1D,1D,2D
3,LT.Kiet,505,1D,1D,1D,1D,1D,1D,2D
4,CM HT.Phat,159,1D,1D,1D,1D,1D,1D,2D


In [185]:
locktime_lockbefore_melt = locktime_lockbefore.melt(id_vars=['Store_Name','Store'], \
                                                    value_vars=locktime_lockbefore.columns[2:],
                                                    var_name='THU',
                                                    value_name='NGAYDONHANG')
locktime_lockhour_melt = locktime_lockhour.melt(id_vars=['Store_Name','Store'], 
                                                value_vars=locktime_lockhour.columns[2:],
                                                var_name='THU',
                                                value_name='GIOLOCK')


In [186]:
locktime_merge = pd.merge(locktime_lockhour_melt, locktime_lockbefore_melt, on=['Store_Name','Store','THU'],how='left')

In [187]:
locktime_merge.rename(columns={'Store':'*DEST'},inplace=True)
locktime_merge['*TRANSMODE'] = 'TPTS_TRAICAY_TT'
locktime_merge['GIOLOCK'] = locktime_merge['GIOLOCK'].astype('str').str.replace(' PM','').str.replace(' AM','').apply(lambda x: x[:-3])
locktime_merge['NGAYDONHANG'] = locktime_merge['NGAYDONHANG'].apply(lambda x: x[:-1])
dayofweek_dict = {'Monday':'1', 'Tuesday':'2', 'Wednesday': '3', 'Thursday': '4', 'Friday': '5', 'Saturday': '6', 'Sunday': '7'}
for weekday in dayofweek_dict.keys():
    locktime_merge['THU'] = locktime_merge['THU'].astype('str').str.replace(weekday, dayofweek_dict[weekday])
locktime_merge.drop(columns='Store_Name',inplace=True)
locktime_merge = locktime_merge[~locktime_merge['*DEST'].isin([2173, 799])]
locktime_merge = locktime_merge.sort_values(by='*DEST',ascending=True)


In [188]:
locktime_merge

,*DEST,THU,GIOLOCK,NGAYDONHANG,*TRANSMODE
1953,112,5,15:30,2,TPTS_TRAICAY_TT
3480,112,1,11:15,2,TPTS_TRAICAY_TT
1444,112,4,15:30,2,TPTS_TRAICAY_TT
2971,112,7,15:30,2,TPTS_TRAICAY_TT
2462,112,6,15:30,2,TPTS_TRAICAY_TT
...,...,...,...,...,...
1868,69068,5,10:30,1,TPTS_TRAICAY_TT
2886,69068,7,10:45,1,TPTS_TRAICAY_TT
2377,69068,6,10:30,1,TPTS_TRAICAY_TT
341,69068,2,10:30,1,TPTS_TRAICAY_TT


In [189]:
locktime_merge[['*TRANSMODE','*DEST','THU','GIOLOCK','NGAYDONHANG']].to_csv('TPTS_DAYOFWEEK.csv',index=False)